# 🃏 Poker Card Image Classifier with PyTorch & EfficientNet

In this notebook, I build a deep learning model to classify images of poker playing cards using a convolutional neural network (CNN).  
Instead of designing the CNN from scratch, I use **EfficientNet-B0** as a pretrained backbone (via the `timm` library) to leverage transfer learning for better accuracy and faster convergence.

**Key points:**

✅ Custom PyTorch `Dataset` wrapping poker card images organized in folders  
✅ Transfer learning with `efficientnet_b0` as the feature extractor  
✅ Custom classifier head on top of EfficientNet  
✅ Training, validation, and testing loops with loss visualization  
✅ Evaluation on a separate test set to report final accuracy

This project was built and run in a Kaggle notebook, using the poker card image dataset stored in:
- `/kaggle/input/cards-image-datasetclassification/train/`
- `/kaggle/input/cards-image-datasetclassification/valid/`
- `/kaggle/input/cards-image-datasetclassification/test/`

**Goal:**  
> Automatically recognize and classify 53 different poker playing cards from images, exploring CNN architectures and transfer learning as part of my deep learning practice.

---

*Built with: PyTorch, torchvision, timm, numpy, matplotlib & tqdm.*

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

## 🗂 Custom dataset

We wrap our poker card images in a custom PyTorch `Dataset` using `ImageFolder`, so data is ready for loading.

In [ ]:
class PokerCardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform = transform)
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    @property
    def classes(self):
        return self.data.classes

## 🎯 Mapping class indices

Get the mapping from numeric labels to poker card names.

In [ ]:
# Get a dictionary associating target values with folder names
data_dir = '/kaggle/input/cards-image-datasetclassification/train'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

## 🧠 Model: EfficientNet + classifier

We use `efficientnet_b0` pretrained on ImageNet as feature extractor, and add a linear classifier for 53 poker card classes.

In [ ]:
class CardClassifer(nn.Module):
    def __init__(self, num_classes = 53):
        super(CardClassifer, self).__init__()

        self.base_model = timm.create_model('efficientnet_b0', pretrained = True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
    
        enet_out_size = 1280
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output

## 🔄 Data transforms & loaders

Resize images to 128×128 and prepare DataLoader objects for training, validation, and testing.

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

train_folder = '../input/cards-image-datasetclassification/train/'
valid_folder = '../input/cards-image-datasetclassification/valid/'
test_folder = '../input/cards-image-datasetclassification/test/'

train_dataset = PokerCardDataset(train_folder, transform = transform)
val_dataset = PokerCardDataset(valid_folder, transform = transform)
test_dataset = PokerCardDataset(test_folder, transform = transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)



## ⚙️ Training config

Set device (CPU or GPU), define loss function, optimizer, and number of epochs.

In [ ]:
num_epochs = 8
train_losses, val_losses = [],[]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = CardClassifer(num_classes = 53)
model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc = 'Training loop'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = loss_func(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    model.eval()
    running_loss= 0.0
    with torch.no_grad():
        for images, label in tqdm(val_loader, desc = 'Validaton loop'):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_func(output, labels)
            running_loss += loss.item()*labels.size(0)

    val_loss = running_loss/ len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {train_loss}. Validation Loss: {val_loss}")

## 📈 Plotting losses

Visualize how training and validation loss change over epochs.

In [ ]:
plt.plot(train_losses, label = 'Training Loss')
plt.plot(val_losses, label = 'Validation Loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()

## ✅ Evaluate on test set

Measure final accuracy on unseen test data.

In [ ]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc = 'Testing Loop'):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct/total
print(f"Validation Accuracy: {accuracy:.4f}")

        